In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import math
import re
import nltk

from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer




def ROUGE2_pre(g, h):

    ac_words1 = g.split(' ')
    pro_words1 = h.split(' ')


    ac_words = list(nltk.bigrams(ac_words1))
    pro_words = list(nltk.bigrams(pro_words1))


    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
               if x == k:
                    count= count + 1
                    break
    elif len(pro_words)>len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break

    pre_rouge_2 = count / len(pro_words)

    return pre_rouge_2



def ROUGE2_re(g, h):


    ac_words1 = g.split(' ')
    pro_words1 = h.split(' ')


    ac_words = list(nltk.bigrams(ac_words1))
    pro_words = list(nltk.bigrams(pro_words1))

    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
                if x == k:
                    count = count + 1
                    break
    elif len(pro_words) > len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break

    # Storing the final accuracy of Rouge-1
    if(len(ac_words)==0):
      re_rouge_2 = count / 1
    else:
      re_rouge_2 = count / len(ac_words)

    return re_rouge_2




def ROUGE1_pre(g, h):

    ac_words = g.split(' ')
    pro_words = h.split(' ')


    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
               if x == k:
                    count= count + 1
                    break
    elif len(pro_words)>len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break


    pre_rouge_1 = count / len(pro_words)

    return pre_rouge_1


def ROUGE1_re(g, h):

    ac_words = g.split(' ')
    pro_words = h.split(' ')

    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
                if x == k:
                    count = count + 1
                    break
    elif len(pro_words) > len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break

    if(len(ac_words)==0):
      re_rouge_1 = count / 1
    else:
      re_rouge_1 = count / len(ac_words)

    return re_rouge_1




def generate_summary(data):


    data = data.replace('\r\n', ' ')
    data   = data.split('۔')


    rsentencted = ""
    rankedSentences = textrankTfIdf(data)
    orderedSentences = orderSentences(rankedSentences, data, data)
    for ordered in orderedSentences:
        if ordered != "":
            rsentencted = rsentencted + str(ordered) +'۔'
    return rsentencted



def textrankTfIdf(document):

    sentences = document
    bow_matrix = CountVectorizer().fit_transform(sentences)


    normalized = TfidfTransformer().fit_transform(bow_matrix)
    similarity_graph = normalized * normalized.T

    nx_graph = nx.from_numpy_array((similarity_graph))
    scores = nx.pagerank(nx_graph)
    return sorted(((scores[i], s) for i, s in enumerate(sentences)),reverse=True)



def orderSentences(rankedList, data, initSentences):
    index = ['']*len(data)
    # print(rankedList)
    for eachRanked in rankedList[0:int(math.ceil(0.13*len(rankedList)))]:
        sen = eachRanked[1]
        index[data.index(sen)] = initSentences[data.index(sen)]
        # print(data.index(sen))
    return index






df = pd.read_csv('Urdu_7_Books.csv')


X = df.Text
y = df.Khulasay
z = df.Markazi_Khayal

j = 0
precount = 1
presumscore = 0
preavg = 0

resumscore = 0
reavg = 0

f_score = 0
f_sums = 0
f_avg = 0

presumscore2 = 0
preavg2 = 0

resumscore2 = 0
reavg2 = 0

f_score2 = 0
f_sums2 = 0
f_avg2 = 0


for x in X:
    XC = generate_summary(str(' '+X[j]+' '))
    precision = ROUGE1_pre(XC, y[j])
    recall = ROUGE1_re(XC, y[j])

    presumscore = presumscore + precision
    resumscore = resumscore + recall

    preavg = presumscore / precount
    reavg = resumscore / precount

    # ==========ROUGE-2==========================================

    precision2 = ROUGE2_pre(XC, y[j])
    recall2 = ROUGE2_re(XC, y[j])
    # print(XC)

    presumscore2 = presumscore2 + precision2
    resumscore2 = resumscore2 + recall2

    preavg2 = presumscore2 / precount
    reavg2 = resumscore2 / precount

    precount = precount + 1
    j = j + 1

f_score = 2 * (preavg * reavg) / (preavg + reavg)

f_score2 = 2 * (preavg2 * reavg2) / (preavg2 + reavg2)

print("average recall is: ")
print(reavg)

print("average Precision is: ")
print(preavg)

print("f-measure average: ")
print(f_score)

print("for ROUGE 2")

print("ROUGE-2 average Precision is: ")
print(preavg2)

print("AROUGE-2 average recall is: ")
print(reavg2)

print("ROUGE-2 f-measure average: ")
print(f_score2)


average recall is: 
0.723335812318047
average Precision is: 
0.48963337240031213
f-measure average: 
0.5839708998797685
for ROUGE 2
ROUGE-2 average Precision is: 
0.20664836225898273
AROUGE-2 average recall is: 
0.33047871274879526
ROUGE-2 f-measure average: 
0.2542894891306918
